In [5]:
import pandas as pd
import os
from pathlib import Path

def analyze_sentence_overlap(train_file, case_study_folder):
    """
    Analyze overlap between training set sentences and case study sentences.
    
    Parameters:
    - train_file: Path to training set Excel file
    - case_study_folder: Path to folder containing case study Excel files
    """
    
    # Load training set
    print("Loading training set...")
    train_df = pd.read_excel(train_file)
    
    # Find sentence column (flexible column name matching)
    sentence_cols = [col for col in train_df.columns 
                     if 'sentence' in col.lower() or 'message' in col.lower()]
    if not sentence_cols:
        raise ValueError("Could not find sentence/message column in training set")
    
    train_sentences = set(train_df['sentence'].dropna().str.strip().str.lower())
    print(f"Training set: {len(train_sentences)} unique sentences")
    
    # Load case study files
    print(f"\nLoading case study files from: {case_study_folder}")
    case_study_files = list(Path(case_study_folder).glob("*.xlsx")) + \
                       list(Path(case_study_folder).glob("*.xls"))
    
    if not case_study_files:
        raise ValueError(f"No Excel files found in {case_study_folder}")
    
    print(f"Found {len(case_study_files)} case study files\n")
    
    # Analyze each file
    all_case_sentences = set()
    file_details = []
    
    for file_path in case_study_files:
        df = pd.read_excel(file_path)
        
        # Find sentence column
        sentence_cols = [col for col in df.columns 
                        if 'sentence' in col.lower() or 'message' in col.lower()]
        if not sentence_cols:
            print(f"Warning: No sentence column found in {file_path.name}, skipping...")
            continue
        
        # Get unique sentences from this file
        file_sentences = set(df['sentence'].dropna().str.strip().str.lower())
        all_case_sentences.update(file_sentences)
        
        # Calculate overlap for this file
        in_train = len(file_sentences.intersection(train_sentences))
        not_in_train = len(file_sentences) - in_train
        coverage = (in_train / len(file_sentences) * 100) if file_sentences else 0
        
        file_details.append({
            'Flight Log': file_path.name,
            'Total Unique': len(file_sentences),
            'In Training': in_train,
            'Novel': not_in_train,
            'Coverage (%)': round(coverage, 1)
        })
    
    # Overall statistics
    total_case_unique = len(all_case_sentences)
    in_training = len(all_case_sentences.intersection(train_sentences))
    not_in_training = total_case_unique - in_training
    overall_coverage = (in_training / total_case_unique * 100) if total_case_unique else 0
    
    # Find novel sentences
    novel_sentences = all_case_sentences - train_sentences
    
    # Print results
    print("=" * 80)
    print("OVERALL ANALYSIS")
    print("=" * 80)
    print(f"Training set size:           {len(train_sentences)} sentences")
    print(f"Case study unique sentences: {total_case_unique} sentences")
    print(f"In training set:             {in_training} sentences ({overall_coverage:.1f}%)")
    print(f"Novel (not in training):     {not_in_training} sentences ({100-overall_coverage:.1f}%)")
    print()
    
    # Print per-file details
    print("=" * 80)
    print("PER-FLIGHT ANALYSIS")
    print("=" * 80)
    details_df = pd.DataFrame(file_details)
    print(details_df.to_string(index=False))
    print()
    
    # Print sample of novel sentences
    if novel_sentences:
        print("=" * 80)
        print(f"NOVEL SENTENCES (first 20 of {len(novel_sentences)})")
        print("=" * 80)
        for i, sentence in enumerate(list(novel_sentences)[:20], 1):
            print(f"{i}. {sentence}")
        
        if len(novel_sentences) > 20:
            print(f"\n... and {len(novel_sentences) - 20} more novel sentences")
    
    # Save results to Excel
    output_file = "overlap_analysis_results.xlsx"
    with pd.ExcelWriter(output_file) as writer:
        # Summary sheet
        summary_df = pd.DataFrame({
            'Metric': [
                'Training Set Size',
                'Case Study Unique Sentences',
                'Sentences in Training',
                'Novel Sentences',
                'Coverage (%)'
            ],
            'Value': [
                len(train_sentences),
                total_case_unique,
                in_training,
                not_in_training,
                round(overall_coverage, 1)
            ]
        })
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        
        # Per-file details
        details_df.to_excel(writer, sheet_name='Per-Flight', index=False)
        
        # Novel sentences
        if novel_sentences:
            novel_df = pd.DataFrame({'Novel Sentences': list(novel_sentences)})
            novel_df.to_excel(writer, sheet_name='Novel Sentences', index=False)
    
    print(f"\nResults saved to: {output_file}")
    
    return {
        'train_size': len(train_sentences),
        'case_study_size': total_case_unique,
        'in_training': in_training,
        'novel': not_in_training,
        'coverage': overall_coverage,
        'file_details': file_details,
        'novel_sentences': list(novel_sentences)
    }

In [6]:
import os
# Example usage
if __name__ == "__main__":
    # Update these paths to your actual files
    train_file = os.path.join('..', 'dataset', 'train_sentence.xlsx')  # Your training set file
    case_study_folder = os.path.join('..', '..', 'Bahan', 'parsed_logs')  # Folder containing your 7 flight logs
    
    # Check if files exist
    if not os.path.exists(train_file):
        print(f"Error: Training file not found: {train_file}")
        print("Please update the 'train_file' variable with your actual training set file path")
    elif not os.path.exists(case_study_folder):
        print(f"Error: Case study folder not found: {case_study_folder}")
        print("Please update the 'case_study_folder' variable with your actual folder path")
    else:
        results = analyze_sentence_overlap(train_file, case_study_folder)

Loading training set...
Training set: 644 unique sentences

Loading case study files from: ..\..\Bahan\parsed_logs
Found 7 case study files

OVERALL ANALYSIS
Training set size:           644 sentences
Case study unique sentences: 78 sentences
In training set:             11 sentences (14.1%)
Novel (not in training):     67 sentences (85.9%)

PER-FLIGHT ANALYSIS
                                       Flight Log  Total Unique  In Training  Novel  Coverage (%)
parsed_DJIFlightRecord_2024-11-10_[03-09-29].xlsx            17            1     16           5.9
parsed_DJIFlightRecord_2025-05-12_[08-01-12].xlsx            35            3     32           8.6
parsed_DJIFlightRecord_2025-05-12_[08-20-56].xlsx            37            2     35           5.4
parsed_DJIFlightRecord_2025-06-27_[03-23-15].xlsx            32            3     29           9.4
parsed_DJIFlightRecord_2025-06-27_[03-27-42].xlsx            30            6     24          20.0
parsed_DJIFlightRecord_2025-06-27_[03-31-40].xls

In [ ]:
import pandas as pd
import os
from pathlib import Path
from drain3 import TemplateMiner
from drain3.template_miner_config import TemplateMinerConfig

def extract_templates(sentences):
    """
    Extract log templates from sentences using Drain3.
    Returns a dictionary mapping original sentences to their templates.
    """
    config = TemplateMinerConfig()
    config.load({
        'DRAIN': {
            'sim_th': 0.9,  # Similarity threshold
            'depth': 5,      # Depth of all leaf nodes
            'max_children': 100
        }
    })
    
    template_miner = TemplateMiner(config=config)
    sentence_to_template = {}
    
    for sentence in sentences:
        result = template_miner.add_log_message(sentence)
        # Use the template with wildcards as the key
        template = result['template_mined']
        sentence_to_template[sentence] = template
    
    return sentence_to_template

def analyze_sentence_overlap_with_templates(train_file, case_study_folder):
    """
    Analyze overlap between training set and case study using template-based matching.
    
    Parameters:
    - train_file: Path to training set Excel file
    - case_study_folder: Path to folder containing case study Excel files
    """
    
    print("=" * 80)
    print("TEMPLATE-BASED OVERLAP ANALYSIS")
    print("=" * 80)
    
    # Load training set
    print("\n[1/5] Loading training set...")
    train_df = pd.read_excel(train_file)
    
    # Find sentence column
    sentence_cols = [col for col in train_df.columns 
                     if 'sentence' in col.lower() or 'message' in col.lower()]
    if not sentence_cols:
        raise ValueError("Could not find sentence/message column in training set")
    
    train_sentences_raw = train_df['sentence'].dropna().str.strip().tolist()
    print(f"   Loaded {len(train_sentences_raw)} sentences")
    
    # Extract templates from training set
    print("\n[2/5] Extracting templates from training set...")
    train_sentence_to_template = extract_templates(train_sentences_raw)
    train_templates = set(train_sentence_to_template.values())
    print(f"   Found {len(train_templates)} unique templates")
    
    # Load case study files
    print(f"\n[3/5] Loading case study files from: {case_study_folder}")
    case_study_files = list(Path(case_study_folder).glob("*.xlsx")) + \
                       list(Path(case_study_folder).glob("*.xls"))
    
    if not case_study_files:
        raise ValueError(f"No Excel files found in {case_study_folder}")
    
    print(f"   Found {len(case_study_files)} case study files")
    
    # Load all case study sentences
    all_case_sentences_raw = []
    file_sentences_map = {}
    
    for file_path in case_study_files:
        df = pd.read_excel(file_path)
        
        # Find sentence column
        sentence_cols = [col for col in df.columns 
                        if 'sentence' in col.lower() or 'message' in col.lower()]
        if not sentence_cols:
            print(f"   Warning: No sentence column found in {file_path.name}, skipping...")
            continue
        
        sentences = df['sentence'].dropna().str.strip().tolist()
        file_sentences_map[file_path.name] = sentences
        all_case_sentences_raw.extend(sentences)
    
    print(f"   Loaded {len(all_case_sentences_raw)} total sentences from case study")
    
    # Extract templates from case study
    print("\n[4/5] Extracting templates from case study sentences...")
    case_sentence_to_template = extract_templates(all_case_sentences_raw)
    case_templates = set(case_sentence_to_template.values())
    print(f"   Found {len(case_templates)} unique templates")
    
    # Perform overlap analysis
    print("\n[5/5] Analyzing overlap...")
    
    # Overall template overlap
    templates_in_train = case_templates.intersection(train_templates)
    novel_templates = case_templates - train_templates
    overall_coverage = (len(templates_in_train) / len(case_templates) * 100) if case_templates else 0
    
    # Per-file analysis
    file_details = []
    for file_name, sentences in file_sentences_map.items():
        file_templates = set(case_sentence_to_template[s] for s in sentences)
        in_train = len(file_templates.intersection(train_templates))
        not_in_train = len(file_templates) - in_train
        coverage = (in_train / len(file_templates) * 100) if file_templates else 0
        
        file_details.append({
            'Flight Log': file_name,
            'Total Unique Templates': len(file_templates),
            'In Training': in_train,
            'Novel': not_in_train,
            'Coverage (%)': round(coverage, 1)
        })
    
    # Print results
    print("\n" + "=" * 80)
    print("OVERALL ANALYSIS (TEMPLATE-BASED)")
    print("=" * 80)
    print(f"Training set templates:      {len(train_templates)}")
    print(f"Case study templates:        {len(case_templates)}")
    print(f"Templates in training:       {len(templates_in_train)} ({overall_coverage:.1f}%)")
    print(f"Novel templates:             {len(novel_templates)} ({100-overall_coverage:.1f}%)")
    print()
    
    # Print per-file details
    print("=" * 80)
    print("PER-FLIGHT ANALYSIS (TEMPLATE-BASED)")
    print("=" * 80)
    details_df = pd.DataFrame(file_details)
    print(details_df.to_string(index=False))
    print()
    
    # Group novel sentences by template
    if novel_templates:
        print("=" * 80)
        print(f"NOVEL TEMPLATES ({len(novel_templates)} templates)")
        print("=" * 80)
        
        novel_template_examples = {}
        for sentence, template in case_sentence_to_template.items():
            if template in novel_templates:
                if template not in novel_template_examples:
                    novel_template_examples[template] = []
                novel_template_examples[template].append(sentence)
        
        for i, (template, examples) in enumerate(sorted(novel_template_examples.items()), 1):
            print(f"\n{i}. Template: {template}")
            print(f"   Examples ({len(examples)} instances):")
            for ex in examples:  # Show up to 3 examples
                print(f"   - {ex}")
            # if len(examples) > 3:
            #     print(f"   ... and {len(examples) - 3} more")
        
        # if len(novel_templates) > 20:
        #     print(f"\n... and {len(novel_templates) - 20} more novel templates")
    
    # Save results to Excel
    output_file = os.path.join("analysis", "template_overlap_analysis_results.xlsx")
    with pd.ExcelWriter(output_file) as writer:
        # Summary sheet
        summary_df = pd.DataFrame({
            'Metric': [
                'Training Set Templates',
                'Case Study Templates',
                'Templates in Training',
                'Novel Templates',
                'Coverage (%)'
            ],
            'Value': [
                len(train_templates),
                len(case_templates),
                len(templates_in_train),
                len(novel_templates),
                round(overall_coverage, 1)
            ]
        })
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        
        # Per-file details
        details_df.to_excel(writer, sheet_name='Per-Flight', index=False)
        
        # Novel templates with examples
        if novel_templates:
            novel_data = []
            for template, examples in sorted(novel_template_examples.items()):
                novel_data.append({
                    'Template': template,
                    'Count': len(examples),
                    'Example 1': examples[0] if len(examples) > 0 else '',
                    'Example 2': examples[1] if len(examples) > 1 else '',
                    'Example 3': examples[2] if len(examples) > 2 else ''
                })
            novel_df = pd.DataFrame(novel_data)
            novel_df.to_excel(writer, sheet_name='Novel Templates', index=False)
        
        # All templates mapping
        template_mapping = []
        for sentence, template in case_sentence_to_template.items():
            template_mapping.append({
                'Sentence': sentence,
                'Template': template,
                'In Training': 'Yes' if template in train_templates else 'No'
            })
        mapping_df = pd.DataFrame(template_mapping)
        mapping_df.to_excel(writer, sheet_name='Sentence-Template Mapping', index=False)
    
    print(f"\n✓ Results saved to: {output_file}")
    
    return {
        'train_templates': len(train_templates),
        'case_study_templates': len(case_templates),
        'templates_in_training': len(templates_in_train),
        'novel_templates': len(novel_templates),
        'coverage': overall_coverage,
        'file_details': file_details
    }


In [7]:

# Update these paths to your actual files
# train_file = "training_set.xlsx"  # Your training set file
# case_study_folder = "case_study_logs"  # Folder containing your 7 flight logs
train_file = os.path.join('..', 'dataset', 'train_sentence.xlsx')  # Your training set file
case_study_folder = os.path.join('..', '..', 'Bahan', 'parsed_logs')  # Folder containing your 7 flight logs
# Check if files exist
if not os.path.exists(train_file):
    print(f"Error: Training file not found: {train_file}")
    print("Please update the 'train_file' variable with your actual training set file path")
elif not os.path.exists(case_study_folder):
    print(f"Error: Case study folder not found: {case_study_folder}")
    print("Please update the 'case_study_folder' variable with your actual folder path")
else:
    try:
        results = analyze_sentence_overlap_with_templates(train_file, case_study_folder)
    except ImportError:
        print("\nError: drain3 package not found!")
        print("Please install it using: pip install drain3")

config file not found: {'DRAIN': {'sim_th': 0.9, 'depth': 5, 'max_children': 100}}


TEMPLATE-BASED OVERLAP ANALYSIS

[1/5] Loading training set...
   Loaded 839 sentences

[2/5] Extracting templates from training set...
   Found 591 unique templates

[3/5] Loading case study files from: ..\..\Bahan\parsed_logs
   Found 7 case study files


config file not found: {'DRAIN': {'sim_th': 0.9, 'depth': 5, 'max_children': 100}}


   Loaded 2158 total sentences from case study

[4/5] Extracting templates from case study sentences...
   Found 52 unique templates

[5/5] Analyzing overlap...

OVERALL ANALYSIS (TEMPLATE-BASED)
Training set templates:      591
Case study templates:        52
Templates in training:       8 (15.4%)
Novel templates:             44 (84.6%)

PER-FLIGHT ANALYSIS (TEMPLATE-BASED)
                                       Flight Log  Total Unique Templates  In Training  Novel  Coverage (%)
parsed_DJIFlightRecord_2024-11-10_[03-09-29].xlsx                      15            1     14           6.7
parsed_DJIFlightRecord_2025-05-12_[08-01-12].xlsx                      30            4     26          13.3
parsed_DJIFlightRecord_2025-05-12_[08-20-56].xlsx                      29            4     25          13.8
parsed_DJIFlightRecord_2025-06-27_[03-23-15].xlsx                      26            4     22          15.4
parsed_DJIFlightRecord_2025-06-27_[03-27-42].xlsx                      26         